#Ambiente

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Datasets

## AmericanasBR

In [5]:
#baixando os datasets
!curl https://www.inf.ufrgs.br/~viviane/DS/B2W-Reviews01_binario5000_TRAIN.csv > B2W-Reviews01_binario5000_TRAIN.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1657k  100 1657k    0     0   609k      0  0:00:02  0:00:02 --:--:--  609k


In [6]:
df_train = pd.read_csv('B2W-Reviews01_binario5000_TRAIN.csv')

In [7]:
classes = df_train.label.unique()
classes

array([0, 1])

# Entendendo as Embeddigns

A bilbioteca [Gensim](https://radimrehurek.com/gensim/models/word2vec.html) permite treinar e usar word embedings.

A versão da biblioteca a ser usada neste notebook é a 4.

Veja diferenças entre versão 3 e 4 neste [link](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).

In [8]:
!pip install gensim==4.3.1
!pip install scipy==1.10.1

In [9]:
from gensim import utils
import gensim.models

In [10]:
gensim.__version__

'4.3.1'

## Treinar embeddings

A biblioteca Gensim permite que você treine as embeddings do seu corpus

https://radimrehurek.com/gensim/models/word2vec.html

In [11]:
# classe para montar o dataset
class PreProcess:
    def __init__(self, docs):
            self.lista_text = docs
    def __iter__(self):
        for line in self.lista_text:
            # assume there's one document per line, tokens separated by whitespace:
            yield utils.simple_preprocess(line) # este método tokeniza e faz algum preprocessamento
            # https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

Alguns parametros do [Word2vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

*   `vector_size` – dimensionalidade dos vetores das palavras.
*   `window` – tamanho do contexto a considerar, por exemplo windows=5 irá considerar as 5 palavras à esquerda e as 5 palavras à direita da palavra atual como a janela de contexto. O modelo tentará então prever a palavra atual dado este contexto.
*   `min_count` – ignora palavras com frequência total menor do que min_count.
*   `sg` – o algoritmo de treinamento: 1 for skip-gram e diferente disto CBOW.





In [12]:
sentences = PreProcess(df_train['text'].values)
# assim treina o modelo usando as configurações padrão e estas especificadas aqui
model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, epochs=20, sg=1)

Quando não se necessita mais do estado completo do modelo treinado (não precisa continuar treinando), `Gensim` permite separar os vetores treinados em [`KeyedVectors`](https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors) possibilitando salvar apenas os vetores e suas chaves (as palavras).



In [13]:
# desta forma acessa somente as palavras e seus vetores
word_vectors = model.wv
word_vectors

In [14]:
# total de palavras e as 10 primeiras
words = list(word_vectors.key_to_index)
print(f'O vocabulario contém {len(words)} palavras')
print(words[0:10])

O vocabulario contém 13905 palavras
['de', 'não', 'produto', 'que', 'muito', 'com', 'do', 'um', 'para', 'da']


In [15]:
# verificando o id de uma palavra:
print('id de entrega:', word_vectors.key_to_index['entrega'])
print('palavra do id 4:', word_vectors.index_to_key[4])

id de entrega: 12
palavra do id 4: muito


In [16]:
# ocorrências de uma palavra:
palavra = 'entrega'
palavra_cnt = word_vectors.get_vecattr(palavra, "count")
print(f'A palavra {palavra} ocorre {palavra_cnt} vezes no dataset')

A palavra entrega ocorre 1770 vezes no dataset


Cada palavra única do corpus é representada por um vetor de tamanho `vector_size`, que corresponde ao número de dimnesões usado no treinamento.

In [17]:
print(f"Embeddings da palavra produto com dimensão {word_vectors['produto'].shape}")
word_vectors['produto']

Embeddings da palavra produto com dimensão (100,)


array([ 2.14594118e-02, -1.43564984e-01,  1.44296084e-02, -4.37251814e-02,
       -5.83881363e-02, -1.87685549e-01,  3.31583321e-01,  4.29584682e-01,
       -3.92939359e-01, -4.36080813e-01, -1.66334748e-01, -4.21091497e-01,
        1.42138436e-01,  1.62833109e-01,  5.27403772e-01,  1.76028222e-01,
        3.24062318e-01,  7.76604712e-02, -3.36791605e-01, -6.72331154e-01,
        1.23839125e-01,  5.06589003e-02,  4.51447189e-01, -7.49421641e-02,
        2.09336296e-01, -3.19438487e-01,  2.21236944e-02,  1.38170421e-02,
       -3.81976038e-01,  2.73362517e-01,  2.67858148e-01,  3.29186954e-02,
       -2.20870203e-03, -4.66568857e-01, -8.88982043e-02,  3.83580983e-01,
        3.21542978e-01,  3.02924395e-01,  1.97532699e-01,  2.40585729e-01,
        1.45721793e-01, -1.17221147e-01, -3.52386743e-01, -7.30322227e-02,
        1.13775581e-01,  3.25957477e-01,  2.92908758e-01,  1.47897219e-02,
        1.42094314e-01, -1.41042799e-01,  1.72983482e-01, -2.80792892e-01,
       -1.11202583e-01, -

In [18]:
# possuem representações diferentes:
print(word_vectors['agua'][0:5])
print(word_vectors['água'][0:5])

[-0.17167191  0.4017774   0.4094458   0.29694277 -0.27416182]
[0.16846634 0.37590247 1.0228361  0.11102752 0.15088724]


Salvando as embeddings treinadas:

In [19]:
# salva o modelo em formato binario do gensim:
model.save("word2vec.model")

In [20]:
# salva em formato texto somete as palavras e seus vetores de embeddings
word_vectors = model.wv
word_vectors.save_word2vec_format("word2vec.txt", binary= False)


## Lendo os vetores

In [21]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('word2vec.txt', binary=False)

In [22]:
word_vectors

In [23]:
print('Total de palavras: ',len(word_vectors))
print('id da palavra água:', word_vectors.key_to_index['água'])
print(word_vectors['água'][0:10])

Total de palavras:  13905
id da palavra água: 206
[ 0.16846634  0.37590247  1.0228361   0.11102752  0.15088724 -0.3395709
  0.3122421   0.75842476 -0.19258109 -0.6397965 ]


In [24]:
#somente os vetores das embeddings:
vectors = word_vectors.vectors
vectors

array([[ 0.01518099, -0.05958486,  0.26219025, ..., -0.29726112,
        -0.03803873,  0.13280226],
       [-0.03155407, -0.09829661,  0.321282  , ..., -0.29171032,
        -0.05862381,  0.18714386],
       [ 0.02145941, -0.14356498,  0.01442961, ..., -0.19220418,
         0.06423458, -0.105435  ],
       ...,
       [ 0.05190526,  0.14438856,  0.1942519 , ..., -0.22856915,
        -0.05480747,  0.09581611],
       [-0.03834829,  0.2486873 ,  0.09278081, ..., -0.29674256,
        -0.04483787,  0.06879474],
       [-0.00699921,  0.06322306,  0.06962314, ..., -0.16607204,
        -0.10333565,  0.03026452]], dtype=float32)

In [25]:
# acessando o vetor da palavra água
print(vectors[206][0:10])

[ 0.16846634  0.37590247  1.0228361   0.11102752  0.15088724 -0.3395709
  0.3122421   0.75842476 -0.19258109 -0.6397965 ]


## Visualizando embeddings

In [26]:
from sklearn.manifold import TSNE
import plotly.express as px

In [29]:
%%time
tsne = TSNE(n_components=3, random_state=0)
projections = tsne.fit_transform(vectors, )
# !pip install umap-learn

# import umap
# umap_model = umap.UMAP(n_components=3, random_state=0)
# projections = umap_model.fit_transform(vectors)


CPU times: user 9min 47s, sys: 929 ms, total: 9min 48s
Wall time: 9min 50s


In [30]:
dfP = pd.DataFrame(projections)
dfP['word'] = words
fig = px.scatter_3d(dfP, x=0, y=1, z=2,hover_data=['word'])
fig.update_traces(marker_size=3)
fig.show()

Também é possível usar o [Embeddings Projector](https://projector.tensorflow.org/) do Tensorflow.

## Usando embeddings já treinadas
Podemos usar word embeddings que já foram treinadas e disponibilizadas

Existem modelos disponíveis no Gensim:

In [31]:
import gensim.downloader as api

In [32]:
disponiveis = api.info()
disponiveis.keys()

dict_keys(['corpora', 'models'])

In [33]:
disponiveis['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [34]:
word_vectors = api.load("glove-wiki-gigaword-100") #128Mb

[==================================================] 100.0% 128.1/128.1MB downloaded


In [35]:
#é uma lista de palavras e seus vetores de embeddinsg treinados por alguém e em algum algoritmo que normlmente é especificado na nomenclatura do arquivo juntamente com o número de dimensões
word_vectors

## Operações com embeddings

A similaridade entre vetores de embeddings é dada pelo cosseno. Quanto mais próximo de 1 mais similar

In [36]:
similarity = word_vectors.similarity('woman', 'man')
similarity

0.8323495

In [37]:
word_vectors.similarity('vehicle', 'car')

0.86308384

In [38]:
similarity = word_vectors.similarity('woman', 'fruit')
similarity

0.2577077

In [39]:
import numpy as np
from numpy.linalg import norm

In [40]:
A = word_vectors['woman']
B = word_vectors['man']

In [41]:
# lembrando que o cosseno é o produto escalar normalizado
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.83234936



queen = (king - man) + woman

In [42]:
#testando o exemplo famoso do artigo do Mikolov sobre word2vec
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7698540687561035)]


In [43]:
result = word_vectors.most_similar(positive=['woman', 'programmer'], negative=['man'], topn=1)
print(result)

[('educator', 0.5853328704833984)]


## Material suplementar - outras embeddings

Glove: http://github.com/stanfordnlp/glove


Word2vec treinado no detalhe no Keras:
https://www.tensorflow.org/tutorials/text/word2vec


Doc2Vec

https://cs.stanford.edu/~quocle/paragraph_vector.pdf

https://alvinntnu.github.io/python-notes/nlp/doc2vec.html

# Exercícios para entregar

Carregue as embeddings indicadas em português para os três exercicios.

In [44]:
#baixando as embeddings do NILC de http://nilc.icmc.usp.br/embeddings
!curl http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip > cbow_s100.zip
!unzip -o cbow_s100.zip
nomearq = 'cbow_s100.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  310M  100  310M    0     0  10.2M      0  0:00:30  0:00:30 --:--:-- 11.0M
Archive:  cbow_s100.zip
  inflating: cbow_s100.txt           


In [47]:
%%time
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(nomearq, binary=False)
print('Carregado: ',nomearq)

Carregado:  cbow_s100.txt
CPU times: user 60 s, sys: 1.41 s, total: 1min 1s
Wall time: 1min


In [48]:
# total de palavras e as 10 primeiras
words = list(word_vectors.key_to_index)
print(f'O vocabulario contém {len(words)}')

O vocabulario contém 929606


## Exercício 1

A polissemia ocorre quando uma mesma palavra possui mais de um significado. Um exemplo de polissemia é a palavra “manga”, que pode ser parte de vestimenta ou uma fruta.

a) Usando a função `most_similar` do Gensim, analise o resultado para a palavra "manga". Escolha outra palavra polissêmica que exista no vocabulário e verifique as palavras mais similares.

In [49]:
# Palavras similares a "manga"
print("Palavras mais similares a 'manga':")
similares_manga = word_vectors.most_similar('manga', topn=10)
for palavra, score in similares_manga:
    print(f"{palavra}: {score:.4f}")

# Outra palavra polissêmica: "banco"
print("\nPalavras mais similares a 'banco':")
similares_banco = word_vectors.most_similar('banco', topn=10)
for palavra, score in similares_banco:
    print(f"{palavra}: {score:.4f}")


Palavras mais similares a 'manga':
lapela: 0.6999
cola: 0.6836
laranja: 0.6765
groselha: 0.6659
capa: 0.6613
lona: 0.6587
argola: 0.6565
maça: 0.6551
gaze: 0.6540
barretina: 0.6517

Palavras mais similares a 'banco':
perk: 0.6580
rosario: 0.6567
cine-theatro: 0.6541
escritório: 0.6217
bc: 0.6214
observatório: 0.5969
comitж: 0.5947
comité: 0.5792
tesouro: 0.5711
edsa: 0.5659


b) O que você observa e qual a sua hipótese para explicar esse comportamento.

> A causa desse comportamento é que modelos de Word2Vec associam cada palavra a um único vetor fixo, independentemente do contexto em que a palavra aparece. Assim, palavras com múltiplos sentidos (polissemia) são representadas de forma ambígua ou imprecisa, refletindo uma média dos contextos em que ocorrem no corpus.

## Exercício 2

a) Faça o exercício para duas palavras:

Escolha uma palavra, um sinônimo e um antônimo da mesma.
Calcule a distância euclideana e a similaridade do cosseno entre a palavra e seu sinônimo e a palavra e seu antônimo.






In [50]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (norm(v1) * norm(v2))

def euclidean_distance(v1, v2):
    return norm(v1 - v2)

# Definindo palavras
palavra = 'feliz'
sinonimo = 'alegre'
antonimo = 'triste'

# Vetores
v_feliz = word_vectors[palavra]
v_alegre = word_vectors[sinonimo]
v_triste = word_vectors[antonimo]

# Distâncias e similaridades
dist_sin = euclidean_distance(v_feliz, v_alegre)
dist_ant = euclidean_distance(v_feliz, v_triste)

sim_sin = cosine_similarity(v_feliz, v_alegre)
sim_ant = cosine_similarity(v_feliz, v_triste)

# Resultados
print(f"Distância Euclidiana (feliz - alegre): {dist_sin:.4f}")
print(f"Distância Euclidiana (feliz - triste): {dist_ant:.4f}")
print(f"Similaridade cosseno (feliz - alegre): {sim_sin:.4f}")
print(f"Similaridade cosseno (feliz - triste): {sim_ant:.4f}")


Distância Euclidiana (feliz - alegre): 4.2465
Distância Euclidiana (feliz - triste): 1.4265
Similaridade cosseno (feliz - alegre): 0.2393
Similaridade cosseno (feliz - triste): 0.8169


b) O que você observa e qual a sua hipótese para explicar esse comportamento.

>

*   "Feliz" está mais distante de "alegre" do que de "triste", tanto na distância euclidiana quanto na similaridade do cosseno.
*   A similaridade cosseno entre "feliz" e "triste" é muito alta (0.81), o que sugere que o modelo entende essas palavras como muito próximas.
*   Em contrapartida, "feliz" e "alegre" (sinônimos) tiveram uma distância maior e menor similaridade.

Antônimos aparecem em contextos semelhantes
Palavras como "feliz" e "triste" costumam aparecer nos mesmos contextos gramaticais ou sintáticos, por exemplo:

*   "Hoje estou muito ___"
*   "Ele parecia ___ durante a festa"

Por isso, os vetores de palavras opostas podem acabar muito próximos, pois Word2Vec aprende por contexto, e não pelo significado.


A similaridade vetorial entre palavras não necessariamente reflete sinonímia ou antonímia semântica, mas sim proximidade de uso contextual.

Por isso, antônimos como "feliz" e "triste" podem parecer próximos, enquanto sinônimos aparentam estar distantes se não compartilham muito contexto explícito no corpus.





## Exercício 3

a) Verifique as palavras mais similares em relação às palavras "enfermeiro" e "enfermeira".

In [51]:
# Verificando as palavras mais similares a "enfermeiro"
print("Palavras mais similares a 'enfermeiro':")
similares_enfermeiro = word_vectors.most_similar('enfermeiro', topn=10)
for palavra, score in similares_enfermeiro:
    print(f"{palavra}: {score:.4f}")

# Verificando as palavras mais similares a "enfermeira"
print("\nPalavras mais similares a 'enfermeira':")
similares_enfermeira = word_vectors.most_similar('enfermeira', topn=10)
for palavra, score in similares_enfermeira:
    print(f"{palavra}: {score:.4f}")


Palavras mais similares a 'enfermeiro':
anestesista: 0.7805
dentista: 0.7740
psicólogo: 0.7639
ortopedista: 0.7517
oftalmologista: 0.7468
pediatra: 0.7338
gastroenterologista: 0.7238
fonoaudiólogo: 0.7215
obstetra: 0.7185
urologista: 0.7118

Palavras mais similares a 'enfermeira':
cabeleireira: 0.8512
psicóloga: 0.8485
prostituta: 0.8362
faxineira: 0.8309
advogada: 0.8302
bibliotecária: 0.8301
cozinheira: 0.8270
menina: 0.8132
mulher: 0.8091
aeromoça: 0.8000


b) Você observa algum viés? Se sim, qual sua hipótese.


> Sim, há viés de gênero nas embeddings.



*   "Enfermeiro" está associado majoritariamente a profissões da área médica: anestesista, pediatra, urologista, etc.
*   "Enfermeira" está associado a profissões femininas estereotipadas e até cargos menos valorizados: prostituta, faxineira, cozinheira, aeromoça.

O viés de gênero surge porque o modelo Word2Vec foi treinado em um corpus que reflete estereótipos sociais presentes nos textos. Como as embeddings aprendem padrões baseados em coocorrência, o modelo acaba replicando associações culturais enviesadas, ligando "enfermeira" a papéis de gênero femininos, mesmo quando semanticamente incorretos.





